+ need to add possible features that appear to have an effect on demographic
# less social factors for now
* Focus on NE us and look at 10 yr span
* maybe work with NE data 2019 again but look at different pollutants
* work with 3 year data for now of counties
* get all dataframes
# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import klib

from IPython.core.display import display

# pd.set_option('display.max_columns', None)
# pd.reset_optio('max_rows')
#np.set_printoptions(threshold=sys.maxsize)

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set(style='ticks', context='talk')

# Load data and constants

In [ ]:
MA, CT, ME, NY, NH, NJ, PA, VT, DE, MD  = [25, 9, 23, 36, 33, 34, 42, 50, 10 , 24]
NORTH_EAST = [MA, CT, ME, NY, NH, NJ, PA, VT, DE, MD]
RI, FL,GA, NC, SC = [44, 12, 13, 37, 45] # midatlantic
AL, KY, MS, TN, WV = [1, 21, 29, 47, 45] # south east
IL, IN, IA, KS, MI,MN = [17, 18, 19, 20, 26, 27] # mid_west
daily_aqi_by_county_2017 = pd.read_csv('../../data/raw/daily_aqi_by_county_2017.csv')
daily_aqi_by_county_2018 = pd.read_csv('../../data/raw/daily_aqi_by_county_2018.csv')
daily_aqi_by_county_2019 = pd.read_csv('../../data/raw/daily_aqi_by_county_2019.csv')

county_demographic = pd.read_csv('../../data/raw/county_cc-est2019-alldata.csv', encoding='latin-1')

unemployment = pd.read_excel('../../data/raw/Unemployment.xls', header=0)
poverty_levels = pd.read_excel('../../data/raw/PovertyEstimates.xls', header=0)

annual_income_by_county = pd.read_csv('../../data/raw/CAINC1__ALL_AREAS_1969_2019.csv', encoding='latin-1')

def remove_col(df, col_name):
    '''Returns a dataframe with removed column or columns from old dataframe'''
    new_df = df.copy()

    if(type(col_name) != str and len(col_name) > 1):
        for index in col_name:
            new_df = new_df.drop(str(index), axis=1)
    else:
        new_df = new_df.drop(str(col_name), axis=1)
    return new_df

In [ ]:
annual_aqi_by_county_2019 = pd.read_csv('../../data/raw/annual_aqi_by_county_2019.csv')

In [ ]:
print(daily_aqi_by_county_2019.columns)

print(annual_aqi_by_county_2019.columns)
annual_aqi_by_county_2019.head()
# annual_aqi_by_county_2019.shape
annual_aqi_by_county_2019['Days with AQI'].min()

annual_aqi_by_county_2019.isna().sum()
annual_aqi_by_county_2019.head()

In [ ]:
hazard = annual_aqi_by_county_2019['Hazardous Days'] != 0

annual_aqi_by_county_2019.loc[hazard]

* seems that pm5 and pm10 are common for days where aqi is above average

In [ ]:
unemployment.columns
unemployment.head()
unemployment.shape
unemployment
# really need to clean this up looks like data from 2000 to 2019



In [ ]:
poverty_levels # poverty levels in 2019

In [ ]:
pd.set_option('display.max_columns', 100)

poverty_levels.iloc[3:]

In [ ]:
saipe = pd.read_excel('../../data/raw/saipe_economic/est19all.xls')
saipe.head(6)

In [ ]:
# need to use us gov api to extract county educational attainment levels for 2019
